In [1]:
import os
import inro.modeller as m
from copy import deepcopy

In [62]:
mm = m.Modeller()


NAMESPACE = "tmg.input_output.import_binary_matrix"
name2 = "inro.emme.data.matrix.matrix_transaction"
matrix311 = mm.tool(name2)
import_matrices = mm.tool(NAMESPACE)           
NAMESPACE = "inro.emme.data.matrix.change_matrix_properties"
change_matrix = m.Modeller().tool(NAMESPACE)
NAMESPACE = "inro.emme.matrix_calculation.matrix_calculator"
compute_matrix = mm.tool(NAMESPACE)
NAMESPACE = "inro.emme.data.matrix.create_matrix"
create_matrix = mm.tool(NAMESPACE)
NAMESPACE = "inro.emme.data.matrix.delete_matrix"
delete_matrix = mm.tool(NAMESPACE)
eb = m.Modeller().emmebank

inputFolder = r"C:\Users\BCRANE\Documents\2041 Committed OL\Task 10" #Committed with Ontario Line
#inputFolder = r"C:\Users\BCRANE\Documents\2041 Committed Plus\Task 10"
#inputFolder = r"C:\Users\BCRANE\Documents\2041 Base Case Demand Matrices\Task 10"
#inputFolder = r"C:\Users\BCRANE\Documents\2016 GGH Plan\Task 10"
scen = mm.scenario
year = "2041C_OL"
#year = "2041CP"
#year = "2041C"
#year = "2016"
#finalDest = "md53"

calcSpec = {
    "expression": None,
    "result": None,
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": None,
        "destinations": None
    },
    "type": "MATRIX_CALCULATION"}

In [9]:
autoFile = os.path.join(inputFolder,"skim.am_peak.auto.sov.time.mdf")
autoTime = eb.available_matrix_identifier('FULL')
import_matrices(4, autoTime[2:] ,autoFile,mm.scenario, "newskim.am_peak.auto.sov.time")
change_matrix(matrix = autoTime,
              matrix_name = "comOL.am_peak.auto.sov.time" + year,
              matrix_description = "Jan Peak Auto SOV Time" + year)

autodemFile = os.path.join(inputFolder,"trips_auto_peak.mdf")
autoDem = eb.available_matrix_identifier('FULL')
import_matrices(4, autoDem[2:] ,autodemFile,mm.scenario, "newtrips_auto_peak" + year)
change_matrix(matrix = autoDem,
              matrix_name = "comOL_newtrips_auto_peak" + year,
              matrix_description = "Trips Auto Peak" + year)

transitdemFile = os.path.join(inputFolder,"trips_transit_peak.mdf")
transitDem = eb.available_matrix_identifier('FULL')
import_matrices(4, transitDem[2:] ,transitdemFile,mm.scenario, "newtrips_transit_peak" + year)
change_matrix(matrix = transitDem,
              matrix_name = "comOL_newtrips_transit_peak" + year,
              matrix_description = "Trips Transit Peak" + year)

demand = eb.available_matrix_identifier('FULL')
create_matrix(matrix_id = demand,
              matrix_name = "comOL_new_at_tripsatpeak_" + year,
              matrix_description = "P & A Trips Trans Auto Peak " + year )
demCalc = deepcopy(calcSpec)
demCalc["expression"] = autoDem +" + " + transitDem
demCalc["result"] = demand
compute_matrix(demCalc)


{'average': 1.441875,
 'expression_type': 'FULL',
 'maximum': 16438.552734,
 'maximum_at': {'destination': 8885, 'origin': 8885},
 'minimum': 0.0,
 'minimum_at': {'destination': 1034, 'origin': 1001},
 'num_evaluations': 12327121,
 'result_type': 'FULL',
 'sum': 17774166.0}

In [67]:
timeMatrices = ["mf101","mf102","mf103","mf104"] # 311 files have to be hardcoded - could fix this by reading the files - look at sixth line
#timeMatrices = ["mf81","mf83"]
for root, dirs, files in os.walk(os.path.join(inputFolder,"TransitTime")):
    for matrix_file in files:
        if matrix_file.endswith(".mdf"):
            matrix_number = eb.available_matrix_identifier('FULL')
            timeMatrices.append(matrix_number)
            import_matrices(4, matrix_number[2:] ,os.path.join(root, matrix_file),mm.scenario, matrix_file[:-4] )
            change_matrix(matrix = matrix_number,
                          matrix_name = matrix_file[:-4],
                          matrix_description = matrix_file[:-4])    
        elif matrix_file.endswith(".311"):
            matrix311(transaction_file = os.path.join(root, matrix_file),
                      throw_on_error = True,
                      scenario = mm.scenario)

In [68]:
timeMatrices

['mf101', 'mf102', 'mf103', 'mf104', 'mf2', 'mf4', 'mf81']

In [66]:
new = ['mf81', 'mf2', 'mf83', 'mf4', 'mf82', 'mf84']
for mat in new:
    delete_matrix(mat,eb)

In [57]:
minExpression = ""
for mat in timeMatrices:
    if timeMatrices.index(mat) == 0:
        minExpression = minExpression + mat
    else:
        minExpression = minExpression + " .min. " + mat
    calc = deepcopy(calcSpec)
    calc["expression"] = "9999"
    calc["result"] = mat
    calc["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "INCLUDE",
                                   "od_values": mat}
    compute_matrix(calc)

timeMat = create_matrix(matrix_id ="mf",
              matrix_name = "super_new_time_transit_peak" + year,
              matrix_description = "Jan Peak Transit Time" + year)

tranTime = timeMat.id

minCalc = deepcopy(calcSpec)
minCalc["expression"] = minExpression
minCalc["result"] = tranTime

compute_matrix(minCalc)

for mat in timeMatrices:
    delete_matrix(mat,eb)
    

filterCalc = deepcopy(calcSpec)
filterCalc["expression"] = "0"
filterCalc["result"] = tranTime
filterCalc["constraint"]["by_value"] = {"interval_min": 9999,
                                   "interval_max": 9999,
                                   "condition": "INCLUDE",
                                   "od_values": tranTime}
compute_matrix(filterCalc)
"""
autofilterCalc = deepcopy(calcSpec)
autofilterCalc["expression"] = "0"
autofilterCalc["result"] = autoTime
autofilterCalc["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "INCLUDE",
                                   "od_values": tranTime}
compute_matrix(autofilterCalc)
"""

'\nautofilterCalc = deepcopy(calcSpec)\nautofilterCalc["expression"] = "0"\nautofilterCalc["result"] = autoTime\nautofilterCalc["constraint"]["by_value"] = {"interval_min": 0,\n                                   "interval_max": 0,\n                                   "condition": "INCLUDE",\n                                   "od_values": tranTime}\ncompute_matrix(autofilterCalc)\n'

In [60]:
timeMat.id

'mf88'

In [42]:
dest_num = eb.available_matrix_identifier('DESTINATION')
create_matrix(matrix_id = dest_num,
              matrix_name = "time_from_station" + year,
              matrix_description = "Min Transit Time from station " + year )

DestinationMatrix(md20)

In [43]:
dest_num

'md20'

In [22]:
pth = r"\\tore-infs01\Models\10039241 - GGH Multimodal Transportation\EMME Databanks\GGH_Multimodal\Database - For Assignment"
export_matrix = mm.tool("inro.emme.data.matrix.export_matrix_to_csv")

In [79]:
time = "mf87" #2041 with pnrknr
#time = "mf79" # 2016 with pnrknr
#time = "mf80" # 2016 wo pnrknr
#time = "mf88" # 2041 wo pnrknr

#year = "2016"
year = "2041C_OL"

In [73]:
#stations = {"Union": 1084,"OshawaGO":2076,"West Harbour":6201,"KitchenerGO":8352,"BramptonGO":4212,"HamiltonGO":6169}
stations = {"Airport": 4378,"Barrie":8848}

In [80]:
for station in stations.keys():
    
    nearby = str(stations[station])
    filt = deepcopy(calcSpec)
    filt["expression"] = time
    filt["result"] = dest_num
    filt["constraint"]["by_zone"] = {"origins": nearby, "destinations": "all"}
    filt["aggregation"]["origins"] = "+"
    compute_matrix(filt)
    
    if year == "2016":
        scen = 109
    else:
        scen = 203
    
    export_matrix(matrices = dest_num, export_path = pth, scenario = eb.scenario(scen))
    
    os.rename(os.path.join(pth,dest_num + ".csv"),os.path.join(pth,year + "_from_" + station + "_" + nearby + ".csv"))
    

In [29]:
filt

{'aggregation': {'destinations': None, 'origins': '+'},
 'constraint': {'by_value': None,
  'by_zone': {'destinations': 'all', 'origins': '2102'}},
 'expression': 'mf87',
 'result': 'md4',
 'type': 'MATRIX_CALCULATION'}

In [35]:
#demand = "mf27" #2016
#demand = "mf22" #2041C
demand = "mf17"#2041CP

#tranTime = "mf36" #2016
#tranTime = "mf26" #2041C
#tranTime = "mf21" #2041CP

#tranTime = "mf51" #2016 No Filter
#tranTime = "mf48" #2041C No Filter
tranTime = "mf55" #2041CP No Filter

#autoTime = "mf28" #2016
#autoTime = "mf18" #2041C
#autoTime = "mf14" #2041CP

#autoTime = "mf45" #2016 No Filter
#autoTime = "mf44" #2041C No Filter
autoTime = "mf49" #2041CP No Filter


In [14]:
d1 = "md31"
d2 = "md32"
d3 = "md33"
d4 = "md34"
d5 = "md35"
#d6 = finalDest

"""
create_matrix(matrix_id = d1,
              matrix_name = "o_newtimedem_",
              matrix_description = "Time Demand Dest Matrix Transit")

create_matrix(matrix_id = d2,
              matrix_name = "autotransdem_",
              matrix_description = "Demand Dest Matrix  Transit and Auto")

create_matrix(matrix_id = d3,
              matrix_name = "newtimefinal_",
              matrix_description = "New Final Weighted Time Transit Dest Matrix")

create_matrix(matrix_id = d4,
              matrix_name = "autonewtimedem_",
              matrix_description = "Time Demand Dest Matrix Auto")

create_matrix(matrix_id = d5,
              matrix_name = "autonewtimefinal_",
              matrix_description = "New Final Weighted Time Auto Dest Matrix")

#create_matrix(matrix_id = d6,
#              matrix_name = "newAutoTransitRatio",
#              matrix_description = "New Final Auto Transit Ratio Committed Plus")
"""
calc1 = deepcopy(calcSpec)
calc1["expression"] = "(" + tranTime + "/" + autoTime+ ")*" + demand
calc1["result"] = d1
calc1["aggregation"]["origins"] = "+"
calc1["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": autoTime}

compute_matrix(calc1)

calc2 = deepcopy(calcSpec)
calc2["expression"] = demand 
calc2["result"] = d2
calc2["aggregation"]["origins"] = "+"
calc2["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": tranTime}

compute_matrix(calc2)

calc3 = deepcopy(calcSpec)
calc3["expression"] = d1 + "/" + d2
calc3["result"] = d3
calc3["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": d2}
compute_matrix(calc3)
"""
calc4 = deepcopy(calcSpec)
calc4["expression"] = demand + "*" + autoTime
calc4["result"] = d4
calc4["aggregation"]["origins"] = "+"
compute_matrix(calc4)

calc5 = deepcopy(calcSpec)
calc5["expression"] = d4 + "/" + d2
calc5["result"] = d5
calc5["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0,
                                   "condition": "EXCLUDE",
                                   "od_values": d2}
compute_matrix(calc5)
"""
"""
calc6 = deepcopy(calcSpec)
calc6["expression"] = d5 + "/" + d3
calc6["result"] = d6
calc6["constraint"]["by_value"] = {"interval_min": 0,
                                   "interval_max": 0.1,
                                   "condition": "EXCLUDE",
                                   "od_values": d3}

create_matrix(matrix_id = d6,
              matrix_name = "newAutoTransitRatio" + year,
              matrix_description = "New Final Auto Transit Ratio Committed Plus " + year)
compute_matrix(calc6)
"""

'\ncalc6 = deepcopy(calcSpec)\ncalc6["expression"] = d5 + "/" + d3\ncalc6["result"] = d6\ncalc6["constraint"]["by_value"] = {"interval_min": 0,\n                                   "interval_max": 0.1,\n                                   "condition": "EXCLUDE",\n                                   "od_values": d3}\n\ncreate_matrix(matrix_id = d6,\n              matrix_name = "newAutoTransitRatio" + year,\n              matrix_description = "New Final Auto Transit Ratio Committed Plus " + year)\ncompute_matrix(calc6)\n'